<a href="https://colab.research.google.com/github/parthchittawar/Machine-Learning/blob/main/Comment_toxicity_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Comment toxicity
detects toxicity level of the comments

In [ ]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np

In [ ]:
tf

<module 'tensorflow' from '/usr/local/lib/python3.7/dist-packages/tensorflow/__init__.py'>

In [ ]:
#df = os.path.join("jigsaw-toxic-comment-classification", "train.csv")

In [ ]:
df = pd.read_csv('train.csv', error_bad_lines=False, engine="python")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
Skipping line 116559: unexpected end of data


#Preprocess

In [ ]:
from tensorflow.keras.layers import TextVectorization

In [ ]:
TextVectorization

keras.layers.preprocessing.text_vectorization.TextVectorization

In [ ]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [ ]:

df['comment_text']

0         Explanation\nWhy the edits made under my usern...
1         D'aww! He matches this background colour I'm s...
2         Hey man, I'm really not trying to edit war. It...
3         "\nMore\nI can't make any real suggestions on ...
4         You, sir, are my hero. Any chance you remember...
                                ...                        
116552    hii...\n\nkindly let me know how to edit it pr...
116553             , also: Favonian is a super-huge faggot.
116554    "\nYour views of the editor's position are dis...
116555    Not just authoritarian, very political. - you ...
116556    Per your request the only dab links that are l...
Name: comment_text, Length: 116557, dtype: object

In [ ]:
df[df.columns[2:]].values

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [ ]:
X = df['comment_text']
y = df[df.columns[2:]].values

In [ ]:
X

0         Explanation\nWhy the edits made under my usern...
1         D'aww! He matches this background colour I'm s...
2         Hey man, I'm really not trying to edit war. It...
3         "\nMore\nI can't make any real suggestions on ...
4         You, sir, are my hero. Any chance you remember...
                                ...                        
116552    hii...\n\nkindly let me know how to edit it pr...
116553             , also: Favonian is a super-huge faggot.
116554    "\nYour views of the editor's position are dis...
116555    Not just authoritarian, very political. - you ...
116556    Per your request the only dab links that are l...
Name: comment_text, Length: 116557, dtype: object

In [ ]:
y


array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [ ]:
MAX_WORDS = 200000  # number of words in the vocab

In [ ]:

vectorizer = TextVectorization(max_tokens=MAX_WORDS,
                               output_sequence_length=1800,
                               output_mode='int')

In [ ]:
X.values

array(["Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",
       "D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)",
       "Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",
       ...,
       '"\nYour views of the editor\'s position are distorted. I don\'t mind using Beevor or any other of the saint trinity, mentioned by you above, as far as using them follows wikipedia guideline ""stay with the facts"". ""mostly relics"" isn\'t fact. It\'s a personal assessment. You seem to push on authority of the trinity, and willing to p

In [ ]:
vectorizer.adapt(X.values)

In [ ]:
vectorizer_text = vectorizer(X.values)

In [ ]:
vectorizer_text

<tf.Tensor: shape=(116557, 1800), dtype=int64, numpy=
array([[   642,     76,      2, ...,      0,      0,      0],
       [177890,     54,   2664, ...,      0,      0,      0],
       [   419,    437,     70, ...,      0,      0,      0],
       ...,
       [    21,    690,      4, ...,      0,      0,      0],
       [    15,     52,   8031, ...,      0,      0,      0],
       [   463,     21,    305, ...,      0,      0,      0]])>

prepare data pipelines

In [ ]:
#MCSHBAP - map, chache, shuffle, batch, prefetch  from_tensor_slices, list_file
dataset = tf.data.Dataset.from_tensor_slices((vectorizer_text, y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)


In [ ]:
batch_X, batch_y = dataset.as_numpy_iterator().next()

In [ ]:
batch_X

array([[  1057,   6025, 131963, ...,      0,      0,      0],
       [    37,     64,     88, ...,      0,      0,      0],
       [    46,    171,     23, ...,      0,      0,      0],
       ...,
       [     7,     20,     58, ...,      0,      0,      0],
       [   430,   3490,     40, ...,      0,      0,      0],
       [   204,    172,     41, ...,      0,      0,      0]])

In [ ]:
int(len(dataset)*.7)

5099

In [ ]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [ ]:
len(train)

5099

In [ ]:
len(val)

1457

In [ ]:
len(test)

728

In [ ]:
train.as_numpy_iterator().next()

(array([[ 1214,    65,    66, ...,     0,     0,     0],
        [  178,    15,    29, ...,     0,     0,     0],
        [12256,    39,     7, ...,     0,     0,     0],
        ...,
        [17331,  5717,  8362, ...,     0,     0,     0],
        [  133,   192,   820, ...,     0,     0,     0],
        [   71,    12,     9, ...,     0,     0,     0]]),
 array([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]]))

# Creating sequential model


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

In [ ]:
y.shape

(116557, 6)

In [ ]:
y[0]

array([0, 0, 0, 0, 0, 0])

In [ ]:
model = Sequential()
model.add(Embedding(MAX_WORDS+1, 32))
model.add(Bidirectional(LSTM(32, activation="tanh")))


model.add(Dense(128, activation="relu"))
model.add(Dense(256, activation="relu"))
model.add(Dense(128, activation="relu"))
model.add(Dense(6, activation="sigmoid"))

In [ ]:
model.compile(loss='BinaryCrossentropy', optimizer="Adam")

In [ ]:

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          6400032   
                                                                 
 bidirectional (Bidirectiona  (None, 64)               16640     
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 6)                 774       
                                                        

In [ ]:
# history = model.fit(train,
#           epochs=1,
#           validation_data=val)

6981/6981 [==============================] - 2941s 420ms/step - loss: 0.0589 - val_loss: 0.0502


In [ ]:
model = tf.keras.models.load_model("comment_toxicity.h5")

#Plotting

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(10, 7))
df = pd.DataFrame(history.history)
plt.plot(df)
plt.show()

NameError: ignored

<Figure size 720x504 with 0 Axes>

# Make Predictions

In [ ]:
input_text = vectorizer("You suck!!")

In [ ]:
batch_X, batch_y = test.as_numpy_iterator().next()

In [ ]:
test

<TakeDataset element_spec=(TensorSpec(shape=(None, 1800), dtype=tf.int64, name=None), TensorSpec(shape=(None, 6), dtype=tf.int64, name=None))>

In [ ]:
input_text

<tf.Tensor: shape=(1800,), dtype=int64, numpy=array([  7, 378,   0, ...,   0,   0,   0])>

In [ ]:
(model.predict(batch_X) > 0.5).astype(int)

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [ ]:
res = model.predict(tf.expand_dims(input_text, 0))

Evaluating the model

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

In [ ]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [ ]:
for batch in test.as_numpy_iterator():
  X_true, y_true = batch
  yhat = model.predict(X_true)

  y_true = y_true.flatten()
  yhat = yhat.flatten()

  pre.update_state(y_true, yhat)
  re.update_state(y_true, yhat)
  acc.update_state(y_true, yhat)

In [ ]:
print(f"Precision {pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy: {acc.result().numpy()} ")

Precision 0.08592776954174042, Recall:0.02691107615828514, Accuracy: 0.06868132203817368 


#Test and Gradio

In [ ]:
!pip install gradio jinja2

     |████████████████████████████████| 2.9 MB 13.7 MB/s 
     |████████████████████████████████| 1.1 MB 55.8 MB/s 
     |████████████████████████████████| 54 kB 3.5 MB/s 
     |████████████████████████████████| 212 kB 70.4 MB/s 
     |████████████████████████████████| 53 kB 2.8 MB/s 
     |████████████████████████████████| 84 kB 4.3 MB/s 
     |████████████████████████████████| 2.0 MB 53.1 MB/s 
     |████████████████████████████████| 253 kB 77.9 MB/s 
     |████████████████████████████████| 144 kB 75.8 MB/s 
     |████████████████████████████████| 94 kB 2.4 MB/s 
     |████████████████████████████████| 271 kB 72.3 MB/s 
     |████████████████████████████████| 58 kB 8.3 MB/s 
     |████████████████████████████████| 10.9 MB 46.2 MB/s 
     |████████████████████████████████| 79 kB 10.2 MB/s 
     |████████████████████████████████| 43 kB 3.0 MB/s 
     |████████████████████████████████| 4.0 MB 53.4 MB/s 
     |████████████████████████████████| 62 kB 1.2 MB/s 
     |██████████████████████

In [ ]:
# model.save('comment_toxicity.h5')

In [ ]:
import gradio as gr

/usr/local/lib/python3.7/dist-packages/paramiko/transport.py:236: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [ ]:
def score_comment(comment):
    vectorized_comment = vectorizer([comment])
    results = model.predict(vectorized_comment)
    
    text = ''
    for idx, col in enumerate(df.columns[2:]):
        text += '{}: {}\n'.format(col, results[0][idx]>0.5)
    
    return text

In [ ]:
interface = gr.Interface(fn=score_comment, 
                         inputs=gr.inputs.Textbox(lines=2, placeholder='Comment to score'),
                        outputs='text')

In [ ]:
interface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://31341.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<fastapi.applications.FastAPI at 0x7f5bd01a0cd0>,
 'http://127.0.0.1:7861/',
 'https://31341.gradio.app')